# Github & Imports

In [ ]:
import os
import sys
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore

from src.config import *
from src.data import *
from src.models.efficientnet import EfficientNetB5Custom, EfficientNetB0, EfficientNetB5, EfficientNetB7, EfficientNetB4
from src.models.resnet18 import ResNet18_v2
from src.models.ensembling import Ensembling
from src.models.simpleCNN import SimpleCNN
from src.utils import *
from src.data import OriginalOAIDataset, DatasetExperiment1
from src.train import train, train_model, test_model
from src.trainers.classification import Classification, ClassificationAdamax
import wandb
# Set random seeds
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)

# Training

In [ ]:
from src.data import DataGatos

BATCH_SIZE = 3
LEARNING_RATE = 0.0001
FACTOR = 0.1
L1 = 0.00
L2 = 0.00
PATIENCE = 5
BETAS=(0.9, 0.999)
LOCAL = True
# Regularización L1 y L2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# wandb 254f72e7bdeec44797ba1b2a91ebbc63900b89f4
# /kaggle/input/aug-oai-capitulo3/augmented_oai
DATASET_PATH = '../../input/aug-oai-capitulo3/augmented_oai'
DATASET_PATH = 'dataset/experimento1/split_oai'
train_dataset = DataGatos(batch_size=BATCH_SIZE).get_dataloader('train')
val_dataset = DataGatos(batch_size=BATCH_SIZE).get_dataloader('val')
#model =ResNet18_v2(n_classes = 5)
model_state = torch.load(r'models\OAI Mendeley\best_model_EfficientNetB5Custom_epoch_2.pt',map_location=torch.device('cpu'), weights_only=False)
model = EfficientNetB5Custom(num_classes=5, pretrained=False)
model.load_state_dict(model_state)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trainer = Classification(model, device, L1=L1, L2=L2, lr=LEARNING_RATE, factor=FACTOR, patience=PATIENCE, betas=BETAS)

In [ ]:
len(train_dataset)

In [ ]:
train(model=model, train_loader=train_dataset, val_loader=val_dataset, trainer=trainer, epochs=2, device=device, wdb= False)

In [ ]:
test_model(model, train_dataset, trainer, device)